# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## Exploring `fastcore.script`

In [ ]:
##| export
from fastcore.all import *
@call_parse
def main(msg:str,     # The message
         upper:bool): # Convert to uppercase?
    'Print `msg`, optionally converting to uppercase'
    print(msg.upper() if upper else msg)

In [ ]:
#| eval: false
! python ../fsdp_qdora_the_explorer/core.py --help

usage: core.py [-h] [--upper] msg

Print `msg`, optionally converting to uppercase

positional arguments:
  msg         The message

options:
  -h, --help  show this help message and exit
  --upper     Convert to uppercase? (default: False)


In [ ]:
#| eval: false
! python ../fsdp_qdora_the_explorer/core.py hello --upper

HELLO


In [ ]:
?call_parse

Signature: call_parse(func=None, nested=False)
Docstring: Decorator to create a simple CLI from `func` using `anno_parser`
File:      ~/miniforge3/envs/default/lib/python3.11/site-packages/fastcore/script.py
Type:      function

In [ ]:
?Param

Signature:      Param(*c, **kw)
Call signature: Param(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function xt>, 'param')
File:           ~/miniforge3/envs/default/lib/python3.11/functools.py
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.

In [ ]:
##| export
@call_parse
def main(msg:Param('The message', str),
         upper:Param('Convert to uppercase?', store_true)):
    'Print `msg`, optionally converting to uppercase'
    print(msg.upper() if upper else msg)

In [ ]:
#| eval: false
! python ../fsdp_qdora_the_explorer/core.py -h

usage: core.py [-h] [--upper] msg

Print `msg`, optionally converting to uppercase

positional arguments:
  msg         The message

options:
  -h, --help  show this help message and exit
  --upper     Convert to uppercase? (default: False)


In [ ]:
help(main)

Help on function main in module __main__:

main(msg: ['param', ('The message', <class 'str'>), {}], upper: ['param', ('Convert to uppercase?', <function store_true>), {}])
    Print `msg`, optionally converting to uppercase



In [ ]:
def f(required:Param('Required param', int),
      a:Param('param 1', bool_arg),
      b:Param('param 2', str)='test'):
    'my_docs'
    pass

In [ ]:
help(f)

Help on function f in module __main__:

f(required: ['param', ('Required param', <class 'int'>), {}], a: ['param', ('param 1', <function bool_arg>), {}], b: ['param', ('param 2', <class 'str'>), {}] = 'test')
    my_docs



## Implementation

In [ ]:
#| export
def main(action:Param('Action', str_enum('settings', 'train'))):
  match action:
    case 'settings': print('Settings')
    case 'train': print('Training')
    case _: print('Unknown')

In [ ]:
?main

Signature: main(action: ['param', ('Action', <enum 'settings'>), {}])
Docstring: <no docstring>
File:      /var/folders/fy/vg316qk1001227svr6d4d8l40000gn/T/ipykernel_85048/3984288533.py
Type:      function

In [ ]:
help(main)

Help on function main in module __main__:

main(action: ['param', ('Action', <enum 'settings'>), {}])



In [ ]:
main('settings')

Settings


In [ ]:
main('lobby')

Unknown


In [ ]:
main

In [ ]:
#| export
def settings(
    world_size: Param('Number of GPUs to use. -1 = all available GPUs.', int) = -1,
    train_type: Param("Training type", choices=["full", "lora", "qlora", "custom_qlora", "custom_lora", "hqq_lora", "hqq_dora", "bnb_dora", "bnb_llama_pro", "hqq_llama_pro"]) = "qlora",
    llama_pro_path: Param('Path to the quantized llama pro model', str) = None,
    batch_size: Param('Batch size per GPU. Effective BS = batch_size * world_size * gradient_accumulation_steps', int) = 1,
    context_length: Param('Max length of input sequence (in tokens)', int) = 512,
    gradient_accumulation_steps: Param('How many steps to accumulate gradients over (increases effective batch size)', int) = 1,
    num_epochs: Param('How many epochs of training to do', int) = 1,
    dataset: Param("Dataset to use", choices=["alpaca", "alpaca_sample", "dummy", "guanaco", "sql", "orca_math"]) = "alpaca_sample",
    dataset_samples: Param('Number of samples in an epoch if using "alpaca_sample" or "dummy" dataset', int) = 512,
    sharding_strategy: Param("Sharding strategy for FSDP", choices=["full_shard", "shard_grad_op", "ddp", "hybrid_full_shard", "hybrid_shard_grad_op"]) = "full_shard",
    use_gradient_checkpointing: Param('Use FSDP\'s activation checkpointing', bool) = True,
    reentrant_checkpointing: Param('Use re-entrant autograd activation checkpointing. Setting to True can use less GPU memory with BNB QLoRA', bool) = False,
    use_cpu_offload: Param('Use FSDP\'s CPU offloading', bool) = True,
    use_activation_cpu_offload: Param('Use FSDP\'s activation CPU offloading', bool) = False,
    low_memory: Param('Load one copy of the model into CPU memory before sharding with FSDP. For QLoRA, quantizes each layer individually on GPU before placing on CPU.', bool) = True,
    no_sync: Param('Prevent gradient sync until update step. Likely uses more memory. Required for `use_cpu_offload` and `gradient_accumulation_steps > 1`', bool) = False,
    precision: Param("Training precision", choices=["fp32", "bf16", "fp16_autocast", "bf16_autocast", "bf16_buffers_autocast"]) = "bf16",
    model_name: Param('Which model to train - e.g. "TinyLlama/TinyLlama-1.1B-Chat-v1.0"', str) = "meta-llama/Llama-2-7b-hf",
    save_model: Param('Save the resulting model', bool) = False,
    output_dir: Param('Output directory to save the final model to', str) = "output",
    lora_rank: Param('LoRA rank for lora/qlora', int) = 64,
    lora_alpha: Param('LoRA alpha for lora/qlora', int) = 16,
    lora_dropout: Param('LoRA dropout for lora/qlora', float) = 0.1,
    lora_target_modules: Param("LoRA target modules", choices=["all", "default"]) = "all",
    verbose: Param('Whether to print extra info for debugging', bool) = False,
    lr: Param('Learning rate', float) = 1e-5,
    apply_gradient_clipping: Param('Apply gradient norm clipping', bool) = False,
    grad_norm: Param('Gradient norm clipping', float) = 0.3,
    wd: Param('Weight decay', float) = 0.1,
    profile_memory: Param('Profile memory usage for the first few batches. Keep false for training. May increase memory usage.', bool) = False,
    optimizer: Param("Optimizer", choices=["adamw", "adam", "sgd", "adadelta"]) = "adamw",
    lr_scheduler: Param("Learning Rate Scheduler", choices=["constant", "linear", "cosine"]) = "constant",
    loading_workers: Param('Number of layers to load and quantize in parallel per GPU. Default of -1 uses heuristics to set worker count.', int) = -1,
    log_to: Param("Where to log output", choices=["tqdm", "wandb", "stdout"]) = "tqdm",
    master_addr: Param('For distributed training', str) = "localhost",
    master_port: Param('For distributed training, must be the same for all processes', str) = "12355",
    seed: Param('Random seed', int) = 42,
    project_name: Param('For wandb logging', str) = "fsdp_qlora",
    name: Param('For wandb logging', str) = None,
    group: Param('For wandb logging', str) = None,
    entity: Param('For wandb logging', str) = None,
    n_bits: Param('passed to hqq', int) = 4,
    profile: Param('Whether to profile with torch.profiler', bool) = False,
    profiling_output: Param('Output file prefix for profiling', str) = "profiles",
    with_stack: Param('Output stacks for profiling. Note that setting export_memory_timeline will automatically export traces since `with_stack` must be true to profile memory.', bool) = False,
    with_shapes: Param('Output shapes for profiling. Can impact performance. Note that setting export_memory_timeline will automatically export traces since `with_shapes` must be true to profile memory.', bool) = False,
    export_trace: Param('Output trace for profiling', bool) = True,
    export_memory_timeline: Param('Output memory timeline for profiling', bool) = False,
    wait_steps: Param('Wait steps when running profiler. Only used if repeat != 0.', int) = 0,
    warmup_steps: Param('Warmup steps when running profiler', int) = 1,
    active_steps: Param('Active steps when running profiler', int) = 2,
    repeat: Param('Number of profiler cycles (wait + warmup + active) if > 0, else repeats forever', int) = 0,
    profiling_frequency: Param('Profiling frequency in steps. Only used if repeat == 0, in which case wait_steps will be set to profiling_frequency - (warmup_steps + active_steps) such that the effective cycle length == profiling_frequency', int) = 10,
    max_steps: Param('Max number of training steps (in units of batches) per epoch. -1 means no max_steps, otherwise training loop breaks after `max_steps` each epoch', int) = -1,
):
  print(f'{locals()}')

In [ ]:
settings()

{'world_size': -1, 'train_type': 'qlora', 'llama_pro_path': None, 'batch_size': 1, 'context_length': 512, 'gradient_accumulation_steps': 1, 'num_epochs': 1, 'dataset': 'alpaca_sample', 'dataset_samples': 512, 'sharding_strategy': 'full_shard', 'use_gradient_checkpointing': True, 'reentrant_checkpointing': False, 'use_cpu_offload': True, 'use_activation_cpu_offload': False, 'low_memory': True, 'no_sync': False, 'precision': 'bf16', 'model_name': 'meta-llama/Llama-2-7b-hf', 'save_model': False, 'output_dir': 'output', 'lora_rank': 64, 'lora_alpha': 16, 'lora_dropout': 0.1, 'lora_target_modules': 'all', 'verbose': False, 'lr': 1e-05, 'apply_gradient_clipping': False, 'grad_norm': 0.3, 'wd': 0.1, 'profile_memory': False, 'optimizer': 'adamw', 'lr_scheduler': 'constant', 'loading_workers': -1, 'log_to': 'tqdm', 'master_addr': 'localhost', 'master_port': '12355', 'seed': 42, 'project_name': 'fsdp_qlora', 'name': None, 'group': None, 'entity': None, 'n_bits': 4, 'profile': False, 'profiling_o

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()